In this notebook, we will perform different steps to train a forecasting model to forecast the total sales of all items across all stores in the next 7 days.

1. Data loading
2. Data preprocessing

    2a. Sales revenue variable
    
    2b. Event_name variable
    
    2c. Create features from 'date' variable

3. Train/ Validation split
4. Baseline model
5. Train and evaluate SGDRegressor model
6. Train and evaluate XGBRegressor model
7. Train and evaluate LinearRegression model
8. Save the best model with joblib

# 1. Data loading

In [1]:
import pandas as pd
import numpy as np

In [2]:
final_df = pd.read_csv('../data/processed/final_df_forecast.csv', low_memory=False)
final_df

ParserError: Error tokenizing data. C error: out of memory

# 2. Data preprocessing

In [ ]:
# Make a copy of final_df and save as final_df_clean
final_df_clean = final_df.copy()

In [ ]:
final_df_clean.isna().sum()

sales_qty            0
date                 0
sell_price    12291876
event_name    43143350
dtype: int64

## 2a. Sales revenue variable

In [ ]:
# Replace NaN values in column 'sell_price' with value 0
final_df_clean['sell_price'] = final_df_clean['sell_price'].replace(np.nan, 0, regex=True)

In [ ]:
# Create a column named sales_revenue
final_df_clean['sales_revenue'] = final_df_clean['sales_qty'] * final_df_clean['sell_price']

In [ ]:
# Drop 'sales_qty', 'sell_price' in combined_df_clean
final_df_clean = final_df_clean.drop(['sales_qty', 'sell_price'], axis=1)
final_df_clean

,date,event_name,sales_revenue
0,2011-01-29,NaN,0.00
1,2011-01-29,NaN,0.00
2,2011-01-29,NaN,0.00
3,2011-01-29,NaN,0.00
4,2011-01-29,NaN,0.00
...,...,...,...
47107045,2015-04-18,NaN,5.76
47107046,2015-04-18,NaN,0.00
47107047,2015-04-18,NaN,0.00
47107048,2015-04-18,NaN,0.00


In [ ]:
# Sort the final_df_clean by date to ensure its chronological order
final_df_clean.sort_values(by='date', inplace=True)

In [ ]:
# Shift the 'sales_revenue' column by 7 days to create the target variable
final_df_clean['target_revenue'] = final_df_clean['sales_revenue'].shift(-7)
final_df_clean

,date,event_name,sales_revenue,target_revenue
0,2011-01-29,NaN,0.00,0.0
20334,2011-01-29,NaN,0.00,0.0
20333,2011-01-29,NaN,0.00,0.0
20332,2011-01-29,NaN,18.90,0.0
20331,2011-01-29,NaN,2.42,0.0
...,...,...,...,...
47086716,2015-04-18,NaN,0.00,NaN
47086715,2015-04-18,NaN,4.52,NaN
47086714,2015-04-18,NaN,0.00,NaN
47086726,2015-04-18,NaN,0.00,NaN


In [ ]:
# Replace NaN values in column 'target_revenue' with value 0
final_df_clean['target_revenue'] = final_df_clean['target_revenue'].replace(np.nan, 0, regex=True)

In [ ]:
# Drop 'sales_revenue' in final_df_clean
final_df_clean = final_df_clean.drop(['sales_revenue'], axis=1)
final_df_clean

,date,event_name,target_revenue
0,2011-01-29,NaN,0.0
20334,2011-01-29,NaN,0.0
20333,2011-01-29,NaN,0.0
20332,2011-01-29,NaN,0.0
20331,2011-01-29,NaN,0.0
...,...,...,...
47086716,2015-04-18,NaN,0.0
47086715,2015-04-18,NaN,0.0
47086714,2015-04-18,NaN,0.0
47086726,2015-04-18,NaN,0.0


## 2b. Event_name variable

In [ ]:
# Drop column 'event_name'
final_df_clean = final_df_clean.drop(['event_name'], axis=1)
final_df_clean

,date,event_name,target_revenue
0,2011-01-29,NotApplicable,0.0
20334,2011-01-29,NotApplicable,0.0
20333,2011-01-29,NotApplicable,0.0
20332,2011-01-29,NotApplicable,0.0
20331,2011-01-29,NotApplicable,0.0
...,...,...,...
47086716,2015-04-18,NotApplicable,0.0
47086715,2015-04-18,NotApplicable,0.0
47086714,2015-04-18,NotApplicable,0.0
47086726,2015-04-18,NotApplicable,0.0


## 2c. Create features from 'date' variable

In [ ]:
# Convert the 'date' column to a datetime type
final_df_clean['date'] = pd.to_datetime(final_df_clean['date'])
final_df_clean

,date,event_name,target_revenue
0,2011-01-29,NotApplicable,0.0
20334,2011-01-29,NotApplicable,0.0
20333,2011-01-29,NotApplicable,0.0
20332,2011-01-29,NotApplicable,0.0
20331,2011-01-29,NotApplicable,0.0
...,...,...,...
47086716,2015-04-18,NotApplicable,0.0
47086715,2015-04-18,NotApplicable,0.0
47086714,2015-04-18,NotApplicable,0.0
47086726,2015-04-18,NotApplicable,0.0


In [ ]:
# Import datetime as dt
import datetime as dt

In [ ]:
# Create features 'year', 'quarter', 'month', 'day_of_week' from column 'date'
final_df_clean['year'] = final_df_clean['date'].dt.year
final_df_clean['quarter'] = final_df_clean['date'].dt.quarter
final_df_clean['month'] = final_df_clean['date'].dt.month
final_df_clean['day_of_week'] = final_df_clean['date'].dt.dayofweek  # 0=Monday, 1=Tuesday, ..., 6=Sunday

final_df_clean

,date,event_name,target_revenue,year,quarter,month,day_of_week
0,2011-01-29,NotApplicable,0.0,2011,1,1,5
20334,2011-01-29,NotApplicable,0.0,2011,1,1,5
20333,2011-01-29,NotApplicable,0.0,2011,1,1,5
20332,2011-01-29,NotApplicable,0.0,2011,1,1,5
20331,2011-01-29,NotApplicable,0.0,2011,1,1,5
...,...,...,...,...,...,...,...
47086716,2015-04-18,NotApplicable,0.0,2015,2,4,5
47086715,2015-04-18,NotApplicable,0.0,2015,2,4,5
47086714,2015-04-18,NotApplicable,0.0,2015,2,4,5
47086726,2015-04-18,NotApplicable,0.0,2015,2,4,5


# 3. Train/ Validation Split

In [ ]:
# Import TimeSeriesSplit from sklearn.model_selection 
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
# Initiate TimeSeriesSplit class with n_splits=5
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
# Split df_clean_sample to train_data and val_data
for train_index, val_index in tscv.split(final_df_clean):
    train_data = final_df_clean.iloc[train_index]
    val_data = final_df_clean.iloc[val_index]

In [ ]:
# Save the target variable of train_data and val_data as y_train and y_val respectively
y_train = train_data['target_revenue']
y_val = val_data['target_revenue']

# 4. Baseline model

In [ ]:
# Find the mean value of target variable and save as y_mean
y_mean = train_data['target_revenue'].mean()

In [ ]:
# Create a numpy array with the same dimensions as for the train_data called y_base filled with this value
y_base = np.full(y_train.shape, y_mean)

In [ ]:
# Import mean_squared_error from sklearn.metrics
from sklearn.metrics import mean_squared_error as mse

In [ ]:
# Print the recall score of this baseline model on the training dataset
print(mse(y_train, y_base, squared=False))

9.006761214320873


# 5. Train and evaluate SGDRegressor model

In [ ]:
# Import Pipeline from sklearn.pipeline, import StandardScaler, OneHotEncoder from sklearn.preprocessing, import SGDRegressor from sklearn.linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor

In [ ]:
# Create a Pipeline called num_transformer with one step that contains StandardScaler
num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

In [ ]:
# Create a list called num_cols that will contain the list of columns that are numeric type
num_cols = ['year', 'quarter', 'month', 'day_of_week']

In [ ]:
# Import ColumnTransformer from sklearn.compose
from sklearn.compose import ColumnTransformer

In [ ]:
# Create a ColumnTransformer called preprocessor containing the following steps

# num_transformer for num_cols
# cat_transformer for cat_cols

preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols)
    ]
)

In [ ]:
# Create a Pipeline called sgd_pipe that contains 2 steps preprocessor and another that instantiate a SGDRegressor with same parameters as previously
sgd_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('sgd', SGDRegressor())
    ]
)

In [ ]:
# Fit sgd_pipe with train_data
sgd_pipe.fit(train_data, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'quarter', 'month',
                                                   'day_of_week']),
                                                 ('cat_cols',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder())]),
                                                  ['event_name'])])),
                ('xgb', SGDRegressor())])

In [ ]:
# Make predictions on train_data and save as y_train_preds
y_train_preds = sgd_pipe.predict(train_data)

In [ ]:
# Make predictions on val_data and save as y_val_preds
y_val_preds = sgd_pipe.predict(val_data)

In [ ]:
# Display the RMSE score on the training set
print(mse(y_train, y_train_preds, squared=False))

8.992413839714242


In [ ]:
# Display the RMSE score on the testing set
print(mse(y_val, y_val_preds, squared=False))

9.943373591599464


# 6. Train and evaluate XGBoost model

In [ ]:
# Import xgboost as xgb
import xgboost as xgb

In [ ]:
# Create a Pipeline called xgb_pipe that contains 2 steps preprocessor and another that instantiate a XGBRegressor with same parameters as previously
xgb_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('xgb', xgb.XGBRegressor())
    ]
)

In [ ]:
# Fit xgb_pipe with train_data
xgb_pipe.fit(train_data, y_train)

c:\Users\this\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'quarter', 'month',
                                                   'day_of_week']),
                                                 ('cat_cols',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder())]),
                                                  ['event_name'])])),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_byno...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [ ]:
# Make predictions on train_data and save as y_train_preds
y_train_preds = xgb_pipe.predict(train_data)

In [ ]:
# Make predictions on val_data and save as y_val_preds
y_val_preds = xgb_pipe.predict(val_data)

In [ ]:
# Display the RMSE score on the training set
print(mse(y_train, y_train_preds, squared=False))

8.985667357418034


In [ ]:
# Display the RMSE score on the testing set
print(mse(y_val, y_val_preds, squared=False))

9.939022724853507


# 7. Train and evaluate LinearRegression model

In [ ]:
# Import LinearRegression from sklearn.linear_model
from sklearn.linear_model import LinearRegression

In [ ]:
# Create a Pipeline called lr_pipe that contains 2 steps preprocessor and another that instantiate a LinearRegression with same parameters as previously
lr_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('lr', LinearRegression())
    ]
)

In [ ]:
# Fit lr_pipe with train_data
lr_pipe.fit(train_data, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_cols',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['year', 'quarter', 'month',
                                                   'day_of_week']),
                                                 ('cat_cols',
                                                  Pipeline(steps=[('one_hot_encoder',
                                                                   OneHotEncoder())]),
                                                  ['event_name'])])),
                ('lr', LinearRegression())])

In [ ]:
# Make predictions on train_data and save as y_train_preds
y_train_preds = lr_pipe.predict(train_data)

In [ ]:
# Make predictions on val_data and save as y_val_preds
y_val_preds = lr_pipe.predict(val_data)

In [ ]:
# Display the RMSE score on the training set
print(mse(y_train, y_train_preds, squared=False))

8.990633242907291


In [ ]:
# Display the RMSE score on the validation set
print(mse(y_val, y_val_preds, squared=False))

9.94076161350763


# 8. Save the best model with joblib

In [ ]:
# Import dump from joblib package and save xgb_pipe (the best model) into models folder
from joblib import dump

dump(xgb_pipe,  '../models/forecasting/xgb_pipe.joblib')

['../models/forecasting/xgb_pipe.joblib']